<a href="https://colab.research.google.com/github/EnesGokceDS/Amazon_Reviews_NLP_Capstone_Project/blob/master/Data_cleaning_and_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **In this study, I will apply data cleaning and feature extraction methods on Amazon Reviews dataset**



---



---



In [ ]:
import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
# mount Google Drive
from os.path import expanduser
gd_path=join(expanduser("~"),'gd')
if not os.path.isdir(gd_path):
    try:
        # load Google Drive
        from google.colab import drive,files
        drive.mount('/drive')
        sp.call('ln -s /drive/My\ Drive '+gd_path, shell=True)
    except:
        print('unable to find Google Drive Folder')
os.chdir(gd_path+'/MachineLearning/NLP_capstone_data')
os.listdir('.')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


['Reviews.csv',
 'Amazon_reviews_processed.csv',
 'Amazon_reviews_cleaned_finalform.csv',
 'Amazon_reviews_after_sentiment.csv',
 'Amazon_reviews_sample_data.csv']

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Reviews.csv')
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [ ]:
df.describe().round(1)

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.0,568454.0,568454.0,568454.0,5.684540e+05
mean,284227.5,1.7,2.2,4.2,1.296257e+09
std,164098.7,7.6,8.3,1.3,4.804331e+07
min,1.0,0.0,0.0,1.0,9.393408e+08
25%,142114.2,0.0,0.0,4.0,1.271290e+09
50%,284227.5,0.0,1.0,5.0,1.311120e+09
75%,426340.8,2.0,2.0,5.0,1.332720e+09
max,568454.0,866.0,923.0,5.0,1.351210e+09


In [ ]:
# Determine how many missing values exist in the collection, in which case you can use .sum() chained onto is.na()
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
Summary,27,0.005
ProfileName,16,0.003
Id,0,0.000
ProductId,0,0.000
UserId,0,0.000
HelpfulnessNumerator,0,0.000
HelpfulnessDenominator,0,0.000
Score,0,0.000
Time,0,0.000
Text,0,0.000


We have small number of missing values. We can drop them completely.

In [ ]:
df= df.dropna()
df.shape

(568411, 10)

# Basic Feature Extraction - 1

Normally, I tried to make data cleaning first. Then, I realized that while making data cleaning, I am losing some of characters that can help data cleaning. Therefore, there will be two part of feature extraction. Here, I will extract features that can't be exracted after data cleaning.

### 1) Number of stopwords

In [ ]:
!pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
df['stopwords'] = df['Text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['Text','stopwords']].head()

,Text,stopwords
0,I have bought several of the Vitality canned d...,21
1,Product arrived labeled as Jumbo Salted Peanut...,12
2,This is a confection that has been around a fe...,42
3,If you are looking for the secret ingredient i...,15
4,Great taffy at a great price. There was a wid...,12


2. Number of Punctuation

In [ ]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['Text'].apply(lambda x: count_punct(x))

In [ ]:
df[['Text','punctuation']].head()

,Text,punctuation
0,I have bought several of the Vitality canned d...,3
1,Product arrived labeled as Jumbo Salted Peanut...,7
2,This is a confection that has been around a fe...,18
3,If you are looking for the secret ingredient i...,5
4,Great taffy at a great price. There was a wid...,5


### 2) Number of hashtag characters

One more interesting feature which we can extract from a review is calculating the number of hashtags or mentions present in it. This also helps in extracting extra information from our text data.

In [ ]:
df['hastags'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['Text','hastags']].head()

,Text,hastags
0,I have bought several of the Vitality canned d...,0
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,0
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,0


In [ ]:
df.hastags.loc[df.hastags != 0].count()

1414

### 3) Number of numerics
Calculate the number of numerics which are present in the tweets can be useful. At least, it doesn't hurt to have such a data!

In [ ]:
df['numerics'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['Text','numerics']].head()

,Text,numerics
0,I have bought several of the Vitality canned d...,0
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,0
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,0


### 4) Number of Uppercase words
Anger or rage is quite often expressed by writing in UPPERCASE words which makes this a necessary operation to identify those words.

In [ ]:
df['upper'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['Text','upper']].head()

,Text,upper
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,2
3,If you are looking for the secret ingredient i...,4
4,Great taffy at a great price. There was a wid...,0




---



---



---



# **Text cleaning techniques**

### Make all text lower case

The first pre-processing step which we will do is transform our reviews into lower case. This avoids having multiple copies of the same words. For example, while calculating the word count, ‘Analytics’ and ‘analytics’ will be taken as different words.

In [ ]:
df['Text'] = df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Text'].head()

0    i have bought several of the vitality canned d...
1    product arrived labeled as jumbo salted peanut...
2    this is a confection that has been around a fe...
3    if you are looking for the secret ingredient i...
4    great taffy at a great price. there was a wide...
Name: Text, dtype: object

### Removing Punctuation

In [ ]:
df['Text'] = df['Text'].str.replace('[^\w\s]','')
df['Text'].head()

0    i have bought several of the vitality canned d...
1    product arrived labeled as jumbo salted peanut...
2    this is a confection that has been around a fe...
3    if you are looking for the secret ingredient i...
4    great taffy at a great price there was a wide ...
Name: Text, dtype: object

### Removal of Stop Words

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['Text'] = df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Text'].sample(10)

524782    time realized tea nothing make nauseated 3 tea...
346063    cider really good option sugar free better col...
187558    cost lot shipped amazing cool pack arrives col...
158972    cookies great flavor love texture filling 6 pk...
274784    sadaf vinegar red wine 32oz seems watered vers...
24060     delicious snacks tastedit amazing serving size...
465953    im already big fan red leaf teas green apple m...
506067    seeing glowing reviews matcha try trying mango...
503433    like spicy ths use without ingredient make hot...
179523    good stuff used find warehouse store dont stoc...
Name: Text, dtype: object

### Removing URLs

In [ ]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

In [ ]:
# remove all urls from df
import re
import string

df['Text'] = df['Text'].apply(lambda x: remove_url(x))

### Remove html tags

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
# remove all html tags from df
df['Text'] = df['Text'].apply(lambda x: remove_html(x))

 ### Removing Emojis
Emojis can be indictor of some emotions that can be related to being customer satisfaction. Unfortunately, we need to remove the emojis in our text analysis

In [ ]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
#Example
remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [ ]:
# remove all emojis from df
df['Text'] = df['Text'].apply(lambda x: remove_emoji(x))

###Remove Emoticons

In previous steps, we have removed emoji. Now, going to remove emoticons. 

***What is the difference between emoji and emoticons?***

*   :-) is an emoticon
*   😜 → emoji.

In [ ]:
!pip install emot

In [ ]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [ ]:
#Example
remove_emoticons("Hello :-)")

'Hello '

In [ ]:
df['Text'] = df['Text'].apply(lambda x: remove_emoticons(x))

### Spell Correction

We’ve all seen tweets with a plethora of spelling mistakes. Our timelines are often filled with hastly sent tweets that are barely legible at times.

In that regard, spelling correction is a useful pre-processing step because this also will help us in reducing multiple copies of words. For example, “Analytics” and “analytcs” will be treated as different words even if they are used in the same sense.

To achieve this we will use the textblob library. 

In [ ]:
from textblob import TextBlob
df['Text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    bought several vitality canned dog food produc...
1    product arrived labelled lumbo halted peanutst...
2    connection around centuries light pillow citie...
3    looking secret ingredient robitussin believe f...
4    great staff great price wide assortment mummy ...
Name: Text, dtype: object

In [ ]:
# We could do some of the cleaning steps as a sum of opreation like this:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
df['Text'] = df.Text.apply(round1)
df.Text

0         bought several vitality canned dog food produc...
1         product arrived labeled jumbo salted peanutsth...
2         confection around centuries light pillowy citr...
3         looking secret ingredient robitussin believe f...
4         great taffy great price wide assortment yummy ...
                                ...                        
568449    great sesame chickenthis good better resturant...
568450    im disappointed flavor chocolate notes especia...
568451    stars small give  one training session tried t...
568452    best treats training rewarding dog good groomi...
568453    satisfied product advertised use cereal raw vi...
Name: Text, Length: 568411, dtype: object

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
df['Text'] = df.Text.apply(round2)
df.Text

0         bought several vitality canned dog food produc...
1         product arrived labeled jumbo salted peanutsth...
2         confection around centuries light pillowy citr...
3         looking secret ingredient robitussin believe f...
4         great taffy great price wide assortment yummy ...
                                ...                        
568449    great sesame chickenthis good better resturant...
568450    im disappointed flavor chocolate notes especia...
568451    stars small give  one training session tried t...
568452    best treats training rewarding dog good groomi...
568453    satisfied product advertised use cereal raw vi...
Name: Text, Length: 568411, dtype: object

Let's check whether the frequent words make sense or not

In [ ]:
freq = pd.Series(' '.join(df['Text']).split()).value_counts()[:20]
freq

br         264689
like       251864
good       195348
one        172306
taste      166572
great      163560
coffee     160180
product    146439
flavor     142442
tea        133094
love       126635
food       123770
would      123356
get        108169
really     100414
dont        95555
much        91906
also        86099
little      83197
use         82743
dtype: int64

# Basic Feature Extraction - 2

###  Number of Words

In [ ]:
df['word_count'] = df['Text'].apply(lambda x: len(str(x).split(" ")))
df[['Text','word_count']].head()

,Text,word_count
0,bought several vitality canned dog food produc...,23
1,product arrived labeled jumbo salted peanutsth...,18
2,confection around centuries light pillowy citr...,40
3,looking secret ingredient robitussin believe f...,18
4,great taffy great price wide assortment yummy ...,13


Again, let's check the data and number of null values

In [ ]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
Id,0,0.0
ProductId,0,0.0
UserId,0,0.0
ProfileName,0,0.0
HelpfulnessNumerator,0,0.0
HelpfulnessDenominator,0,0.0
Score,0,0.0
Time,0,0.0
Summary,0,0.0
Text,0,0.0


### Number of characters

In [ ]:
df['char_count'] = df['Text'].str.len() ## this also includes spaces
df[['Text','char_count']].head()

,Text,char_count
0,bought several vitality canned dog food produc...,164
1,product arrived labeled jumbo salted peanutsth...,136
2,confection around centuries light pillowy citr...,285
3,looking secret ingredient robitussin believe f...,128
4,great taffy great price wide assortment yummy ...,83


### 3) Average Word Length

In [ ]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [ ]:
df['avg_word'] = df['Text'].apply(lambda x: avg_word(x)).round(1)
df[['Text','avg_word']].head()

,Text,avg_word
0,bought several vitality canned dog food produc...,6.2
1,product arrived labeled jumbo salted peanutsth...,6.6
2,confection around centuries light pillowy citr...,6.1
3,looking secret ingredient robitussin believe f...,6.2
4,great taffy great price wide assortment yummy ...,5.5


In [ ]:
df.sample(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
63390,63391,B002IEZJMA,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",0,0,3,1338422400,Strong But Bitter Coffee Taste,illy issimo caffe coffee drink strong coffee f...,34,21,0,0,3,39,240,5.2
97015,97016,B005A1LG7K,A16I008F7QY86T,"G. White ""Booker""",2,2,5,1310947200,Chicago Mix,love popcorn especially cheese popcorn caramel...,14,14,0,0,4,23,151,5.6


**Let's convert the 'time' column to a meaningful format**  

In [ ]:
df['Time'] = pd.to_datetime(df['Time'],unit='s')

In [ ]:
df.sample(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
385196,385197,B005VOOM5E,AKAZT5193KFR1,"L. Samuelson ""L.W. Samuelson""",3,6,4,2012-03-27,Use a French press,poured boiling water pod worked really well sp...,38,11,0,0,2,43,281,5.6
208682,208683,B0019VM5R0,A3KHDOLE6ERXDW,"Carrie Ewoodie Smith ""butterfly""",0,1,5,2010-11-11,MMMMMMMMMMM Good,shortbread fingers cookies pack cookies hi...,12,31,0,0,1,17,91,5.8
59780,59781,B000HIHE2W,A1APZPQ6QQ5SCW,"Kristina ""kristinal""",0,0,1,2011-12-20,CHECK THE EXPIRATION DATE,utilized amazons subscribe save option first t...,29,21,0,0,7,50,347,6.1
181315,181316,B0002SQ4HS,A1GAZ691MAL3R2,Skye Goddess,3,3,5,2011-03-29,Good for your cats!,love cat food four rescue kitties love feel mu...,34,12,0,1,5,44,269,5.3
541719,541720,B001BM4RC8,A265Z64JO5KNSH,Tyler F.,0,0,5,2012-02-28,"Blue, no salt added -- amazing!!",ive subscriber blue chips salt added year love...,35,26,0,0,2,42,265,5.3


Lastly, we don't need 'ProfileName' feature because we already have 'UserId'. Therefore, we can drop 'ProfileName'

In [ ]:
df= df.drop('ProfileName', axis= 1)

In [ ]:
list(df)

['Id',
 'ProductId',
 'UserId',
 'HelpfulnessNumerator',
 'HelpfulnessDenominator',
 'Score',
 'Time',
 'Summary',
 'Text',
 'stopwords',
 'punctuation',
 'hastags',
 'numerics',
 'upper',
 'word_count',
 'char_count',
 'avg_word']

In [ ]:
df.sample(5)

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
305941,305942,B008BY7O9W,A236H3QQ015FKK,0,0,5,2012-09-27,Great Product if Jiff is Excluded from One's Diet,november downloaded livestrong app began maki...,90,25,0,0,19,135,900,6.0
162969,162970,B000R4HQPG,A23GQ96F3P8HG7,0,0,2,2011-01-18,Could be better,im normally fan bigelow green teas blueberry f...,11,13,0,0,1,19,144,6.6
34093,34094,B004N8LMFM,A24Q3XPYXWTQRY,14,15,5,2011-05-09,Entertaining,popin cooking fun enjoyed arrived days early ...,17,16,0,1,1,26,155,5.2
325877,325878,B000G35K6A,A2X36IYVJM49XN,0,4,1,2007-02-13,RECEIVED ONLY ONE BOTTLE -- TWICE,beware sent one bottle contacted amazon said ...,0,12,0,0,37,20,126,5.6
147715,147716,B001D0IZBM,A3S25S0HFAIHPH,0,1,5,2009-02-27,"Emeril's K-cups, Big Easy Bold",sorry say coffee ok didnt buy emerils kcups bi...,8,10,0,1,3,16,76,4.4




---
##**Now, let's apply round 1 and round 2 data cleaning processes on 'Summary' column**


Keep in mind that round1 operations make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.

In [ ]:
df['Summary'] = df.Summary.apply(round1)
df.Summary

0                      good quality dog food
1                          not as advertised
2                        delight says it all
3                             cough medicine
4                                great taffy
                         ...                
568449                   will not do without
568450                          disappointed
568451              perfect for our maltipoo
568452    favorite training and reward treat
568453                           great honey
Name: Summary, Length: 568411, dtype: object

And, round2 operations get rid of some additional punctuation and non-sensical text that was missed the first time around.

In [ ]:
df['Summary'] = df.Summary.apply(round2)
df.Summary

0                      good quality dog food
1                          not as advertised
2                        delight says it all
3                             cough medicine
4                                great taffy
                         ...                
568449                   will not do without
568450                          disappointed
568451              perfect for our maltipoo
568452    favorite training and reward treat
568453                           great honey
Name: Summary, Length: 568411, dtype: object

Let's check whether most frequent words make make. We can add our own stopwords depending on it

In [ ]:
freq = pd.Series(' '.join(df['Text']).split()).value_counts()[:50]
freq

br           264689
like         251864
good         195348
one          172306
taste        166572
great        163560
coffee       160180
product      146439
flavor       142442
tea          133094
love         126635
food         123770
would        123356
get          108169
really       100414
dont          95555
much          91906
also          86099
little        83197
use           82743
time          80813
amazon        78312
buy           76200
tried         75818
best          75371
im            75265
even          74168
price         73469
find          73082
ive           72153
make          71412
well          70149
better        68827
try           67800
eat           66611
dog           66050
first         61370
water         58802
chocolate     58678
bag           55284
used          53932
found         53774
sugar         51845
cup           50632
drink         49927
sweet         49808
made          49755
bought        49699
think         48846
box           48504


#Adding own stopwords

In [ ]:
# Adding common words from our document to stop_words

add_words = ["br",     
"also",     
"im",      
"ive",      
]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

In [ ]:
df['Text'] = df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['Text'].sample(10)

457478    may best pretzels around think taste great gf ...
406219    reason cant find stores youngstown ohio area m...
239872    southwest instant veggie burger mix nutritious...
553778    good beans fair price drinking coffee large vo...
485165    every cat owned loved balls local stores stopp...
350071    calories serving tastes great quite sensitive ...
256684    start saying never enthusiastic fan tinned tun...
424962    huge fan curry dishes excited try disappointed...
131446    mms texture malted milk ball salty crunchy cen...
332619    product delivered within promised time frame s...
Name: Text, dtype: object

In [ ]:
df1= df

In [ ]:
mask = df1.Text.str.endswith('br') 
df1.loc[mask, 'Text'] = df1.loc[mask, 'Text'].str[:-2]

In [ ]:
df1['Text'] = df1['Text'].str.rstrip('tty')

In [ ]:
df1['Text'].apply(lambda x: x[:-2] if x.endswith('tty') else x)

0         bought several vitality canned dog food produc...
1         product arrived labeled jumbo salted peanutsth...
2         confection around centuries light pillowy citr...
3         looking secret ingredient robitussin believe f...
4         great taffy great price wide assortment yummy ...
                                ...                        
568449    great sesame chickenthis good better resturant...
568450    disappointed flavor chocolate notes especially...
568451    stars small give one training session tried tr...
568452    best treats training rewarding dog good groomi...
568453    satisfied product advertised use cereal raw vi...
Name: Text, Length: 568411, dtype: object

In [ ]:
df1.loc[df1.Text.str.endswith('br'), 'Text']

Series([], Name: Text, dtype: object)

In [ ]:
df1.loc[df1.punctuation >= 1000].Text.tolist()

['note amazoncombr pretty please put liver subscribe save beef liver beef available ss liver yet thank muchbr reviewbr cat slowly switching newmans beef products wellness brand cat food great brand still reserved treats occasionally ecstatic totally grainandno fillers organic cat food could feed year old cat prone crystal formation bladder plaque build teeth even though brush outer teeth every eveningbr far tried beef liver liver loves ordered beef today sure hell love toobr product thick concentrated beef liver alone consistency concentrated pate beef liver consistency gelatinous thick beefy oatmeal scoop bowl blend bit hot water product warm fats make gravy prefers eat food bit warm anyhow barely wants nibble wellness brand cat food four years guess means hes one sold cat might unusual cat love beef much cat goes crazy beef especially make steak afford find organic grain beef based line cat food made day one nine lives doubtbr anyway know cat already loves beef beef liver heshe proba

In [ ]:
df.loc[df.punctuation >= 1000].Text.tolist()

['note amazoncombr pretty please put liver subscribe save beef liver beef available ss liver yet thank muchbr reviewbr cat slowly switching newmans beef products wellness brand cat food great brand still reserved treats occasionally ecstatic totally grainandno fillers organic cat food could feed year old cat prone crystal formation bladder plaque build teeth even though brush outer teeth every eveningbr far tried beef liver liver loves ordered beef today sure hell love toobr product thick concentrated beef liver alone consistency concentrated pate beef liver consistency gelatinous thick beefy oatmeal scoop bowl blend bit hot water product warm fats make gravy prefers eat food bit warm anyhow barely wants nibble wellness brand cat food four years guess means hes one sold cat might unusual cat love beef much cat goes crazy beef especially make steak afford find organic grain beef based line cat food made day one nine lives doubtbr anyway know cat already loves beef beef liver heshe proba

In [ ]:
freq = pd.Series(' '.join(df['Text']).split()).value_counts()[:50]
freq

like         251864
good         195348
one          172306
taste        166572
great        163560
coffee       160180
product      146439
flavor       142442
tea          133094
love         126635
food         123770
would        123356
get          108169
really       100414
dont          95555
much          91906
little        83197
use           82743
time          80813
amazon        78312
buy           76200
tried         75818
best          75371
even          74168
price         73469
find          73082
make          71412
well          70149
better        68827
try           67800
eat           66611
dog           66050
first         61370
water         58802
chocolate     58678
bag           55284
used          53932
found         53774
sugar         51845
cup           50632
drink         49927
sweet         49808
made          49755
bought        49699
think         48846
box           48504
way           48236
two           48178
since         46079
day           45310


**Now, let's save this clened processed data as CSV file** 

In [ ]:
df.to_csv('Amazon_reviews_processed.csv', index=False)